In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
def filter_candidates_with_hough(edges, square_candidates, min_lines=500):
    """
    Filter square candidates by detecting many straight lines (likely Sudoku grids).
    
    Args:
        edges: Edge-detected image (result of Canny or similar).
        square_candidates: List of contours representing square candidates.
        min_lines: Minimum number of lines required to consider a candidate as valid.

    Returns:
        List of contours that likely represent Sudoku grids.
    """
    valid_sudoku_candidates = []

    for candidate in square_candidates:
        # Create a mask for the candidate
        mask = np.zeros_like(edges)
        cv2.drawContours(mask, [candidate], -1, 255, thickness=cv2.FILLED)

        # Extract the region of interest (ROI) using the mask
        roi = cv2.bitwise_and(edges, edges, mask=mask)

        # Apply Hough Line Transform
        lines = cv2.HoughLines(roi, 1, np.pi / 180, threshold=100)

        # Count lines and filter
        line_count = len(lines) if lines is not None else 0
        if line_count >= min_lines:
            valid_sudoku_candidates.append(candidate)
        print(f"Detected {line_count} lines for a valid candidate.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return valid_sudoku_candidates
def get_square_candidates(contours, aspect_ratio_epsilon=0.15, binary_search_epsilon=1e-5):
    """
    Process contours to identify valid square-like quadrilaterals.
    
    Args:
        contours: List of contours to process.
        aspect_ratio_epsilon: Allowed deviation for aspect ratio to consider as square.
        binary_search_epsilon: Tolerance for binary search termination in polygon approximation.
0
    Returns:
        List of contours that are valid squares or quadrilaterals.
    """
    valid_squares = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        low, high = 0, 0.5  # Epsilon range (0 to 50% of the perimeter)

        while high - low > binary_search_epsilon:
            epsilon = (low + high) / 2
            approx = cv2.approxPolyDP(contour, epsilon * perimeter, True)

            # Check if the approximation is a quadrilateral
            if len(approx) == 4:
                # Validate aspect ratio
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                if abs(aspect_ratio - 1) < aspect_ratio_epsilon:
                    valid_squares.append(approx)  # Found a valid square
                    break  # Move to the next contour
                else:
                    # Quadrilateral failed aspect ratio test; refine further
                    high = epsilon
            elif len(approx) > 4:
                # Too many sides, increase epsilon to simplify
                low = epsilon
            else:
                # Too few sides, decrease epsilon to capture more detail
                high = epsilon

    return valid_squares
def detect_sudoku_grid(image):
    """
    Detect the Sudoku grid in an image.
    
    Args:
        image: Input image.

    Returns:
        List of valid contours representing Sudoku grids.
    """
    # Step 1: Preprocess the image
    edges=preprocess_image_with_thresholding(image)
    cv2.resize(edges, (800, 600))
    cv2.imshow("edges", edges)
    cv2.waitKey(0)
    # Step 2: Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    draw_all_contours(image, contours, title="All Contours")

    # Step 3: Filter by aspect ratio and shape
    square_candidates = get_square_candidates(contours)
    draw_all_contours(image, square_candidates, title="Square Candidates")

    # Step 4: Filter by Hough lines (lots of grid-like lines indicate a Sudoku grid)
    valid_candidates = filter_candidates_with_hough(edges, square_candidates)
    draw_all_contours(image, valid_candidates, title="Valid Candidates")
    

    return valid_candidates
def order_points(pts):
    """
    Order points in [top-left, top-right, bottom-right, bottom-left] format.
    """
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]  # Top-left
    rect[2] = pts[np.argmax(s)]  # Bottom-right
    rect[1] = pts[np.argmin(diff)]  # Top-right
    rect[3] = pts[np.argmax(diff)]  # Bottom-left
    return rect
def warp_perspective(img, rect):
    """
    Warp the perspective of the image to a 450x450 grid.
    """
    size = 450
    dst = np.array([[0, 0], [size - 1, 0], [size - 1, size - 1], [0, size - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(img, M, (size, size))
    return warped
def draw_all_contours(image, contours, title="Contours"):
    """
    Draw all contours on the image and display them.

    Args:
        image: The original image.
        contours: List of contours to draw.
        title: Title for the displayed window.
    """
    img_copy = image.copy()
    cv2.drawContours(img_copy, contours, -1, (0, 255, 0), 2)
    cv2.imshow(title, img_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def preprocess_image_with_thresholding(image):
    """
    Preprocess the image using adaptive thresholding to prioritize black regions 
    and refine noise, linking white lines effectively.
    
    Args:
        image: Input image (BGR format).
    
    Returns:
        refined: Refined image with linked white lines and noise minimized.
    """
    # Step 1: Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Step 2: Apply adaptive thresholding
    # Adaptive thresholding calculates the threshold for a small region of the image
    # Parameters:
    # - maxValue: Maximum intensity value to use for white regions (255 is standard).
    # - adaptiveMethod: ADAPTIVE_THRESH_GAUSSIAN_C uses a weighted sum of neighborhood values.
    # - thresholdType: THRESH_BINARY converts pixels above the threshold to maxValue.
    # - blockSize: Size of the neighborhood area used to calculate the threshold.
    # - C: Constant subtracted from the threshold. Smaller values make regions darker.
    binary = cv2.adaptiveThreshold(
        gray, 255,  # maxValue for white
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  # Gaussian-weighted thresholding
        cv2.THRESH_BINARY,  # Binary thresholding
        blockSize=15,  # Size of the region for thresholding (must be odd, e.g., 11, 15, 21)
        C=10  # Subtracted constant, larger values make the result darker
    )
    # Step 3: Invert the binary image (makes white regions black and vice versa)
    inverted = cv2.bitwise_not(binary)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 4))
    dilated_vertical = cv2.dilate(inverted, vertical_kernel, iterations=1)
    # Median blur removes salt-and-pepper noise
    median_filtered = cv2.medianBlur(dilated_vertical, 5)
    dilated_vertical = cv2.dilate(median_filtered, vertical_kernel, iterations=1)
    median_filtered = cv2.medianBlur(dilated_vertical, 5)
    # Horizontal dilation (1x4 kernel)
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 1))
    dilated_horizontal = cv2.dilate(median_filtered, horizontal_kernel, iterations=1)
    median_filtered=cv2.medianBlur(dilated_horizontal, 5)
    dilated_horizontal = cv2.dilate(median_filtered, horizontal_kernel, iterations=1)
    median_filtered=cv2.medianBlur(dilated_horizontal, 5)
    return median_filtered
def main_script(image_path):
    image = cv2.imread(image_path)
    
    # Detect the Sudoku grid
    sudoku_candidates = detect_sudoku_grid(image)
    
    if sudoku_candidates:
        # Use the largest valid candidate
        largest_candidate = min(sudoku_candidates, key=cv2.contourArea)
        rect = order_points(np.squeeze(largest_candidate))
    
        # Warp the perspective to get a straightened Sudoku grid
        sudoku_grid = warp_perspective(image, rect)
    
        # Display the resulting grid
        cv2.imshow("Sudoku Grid", sudoku_grid)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No Sudoku grid detected, or no valid candidates found for this path:", image_path)
        cv2.waitKey(0)
def process_sudoku_images_in_folder(folder_path):
    """
    Loop through all images in the folder, process each, and detect Sudoku grids.
    
    Args:
        folder_path: Path to the folder containing Sudoku images.
    """
    # List all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp'))]

    # Process each image file
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        print(f"Processing image: {image_path}")
        main_script(image_path)

In [ ]:
process_sudoku_images_in_folder("small dataset")